### VLLM serve command:

### Codestral-22B-v0.1
``` sh
docker run --rm --gpus all --privileged   --ipc=host  --shm-size=2g --network host -v ~/.cache/huggingface:/root/.cache/huggingface
  -v /home/odedh/SML-For-Debug/models/:/models   vllm/vllm-openai:latest   --model /models/Codestral-22B-v0.1   --use-tqdm-on-load   
--enable-chunked-prefill   --tensor-parallel-size 2

```

### Qwen3-0.6B
``` sh
docker run --rm --gpus all  --privileged   --ipc=host  --shm-size=2g --network host -v ~/.cache/huggingface:/root/.cache/huggingface -v /home/odedh/SML-For-Debug/models/:/models   -e CUDA_VISIBLE_DEVICES=2 vllm/vllm-openai:latest   --model /models/Qwen3-0.6B   --use-tqdm-on-load   --enable-chunked-prefill    --port 8001
```

In [2]:
# !pip install openai

Running a simpel example for the Qwen3-0.6B

In [17]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8001/v1",
    api_key="-",
)
# print the model name
print(client.models.list().data[0].id)

model = client.models.list().data[0].id

code_for_debugging = """
def divide_numbers(a, b):
    return a / b
"""
content = f"Classify this code and ans if there is a bug in it: {code_for_debugging}"


completion = client.chat.completions.create(
    model=model,
    temperature=0.1,
    messages=[
        {"role": "user", "content": content}
    ],
    # extra_body={"guided_choice": ["bug", "no-bug"]},
    
)
print(completion.choices[0].message.content)

/models/Qwen3-0.6B
<think>
Okay, let's see. The user provided a Python function called divide_numbers that takes two parameters, a and b, and returns a divided by b. The question is to classify this code and determine if there's a bug in it.

First, I need to think about what the function does. The function is straightforward: it divides the first number by the second. But wait, what if there's a division by zero? Oh, right! If b is zero, dividing by zero would cause a division by zero error. But in the code given, there's no error checking. So the function as written doesn't handle that case.

But the user is asking if there's a bug in the code. The function is supposed to perform the division, but without error handling. So the code is incomplete. The function doesn't check for division by zero, which is a common bug. However, maybe the user expects me to just classify the code as a simple function and note that division by zero is a potential bug.

Wait, but the question says "class

Codestral-22B-v0.1

In [1]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8001/v1",
    api_key="-",
)
model = client.models.list().data[0].id
print(model)

code_for_debugging = """
def divide_numbers(a, b):
    return a / b
"""
content = f"Classify this code: {code_for_debugging}"


completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": content}
    ],
    extra_body={"guided_choice": ["bug", "no-bug"]},
    
)
print(completion.choices[0].message.content)

/models/Qwen3-0.6B
bug


In [2]:
from pydantic import BaseModel, Field
from enum import Enum
from typing import Optional, List

class BugStatus(str, Enum):
    no_bug = "no_bug"
    has_bug = "has_bug"
    # uncertain = "uncertain"

class Fixability(str, Enum):
    fixable = "fixable"
    not_fixable = "not_fixable"
    unknown = "unknown"

class ClassificationResult(BaseModel):
    has_bug: BugStatus
    bug_type: Optional[str] = Field(default=None, description="Type of bug if present")
    suggested_fix: Optional[str] = Field(default=None, description="Optional fix attempt if model detects a bug")
    confidence_score: float = Field(default=0, ge=0.0, le=1.0)
    fixability: Optional[Fixability] = None
    tested_functions: Optional[List[str]] = None
    rationale: Optional[str] = Field(default=None, description="Short reasoning for classification")

json_schema = ClassificationResult.model_json_schema()


code_snippet = """
def get_api_response_ext(self, http_resp, url='/images', headers={},\n\t\t\t\t\t\t\t body=None, method=None, api=None,\n\t\t\t\t\t\t\t content_type=None):\n\t\tif api is None:\n\t\t\tapi = self.api\n\t\treq = webob.Request.blank(url)\n\t\tfor k, v in headers.iteritems():\n\t\t\treq.headers[k] = v\n\t\tif method:\n\t\t\treq.method = method\n\t\tif body:\n\t\t\treq.body = body\n\t\tif content_type == 'json':\n\t\t\treq.content_type = 'application/json'\n\t\telif content_type == 'octet':\n\t\t\treq.content_type = 'application/octet-stream'\n\t\tres = req.get_response(api)\n\t\tself.assertEqual(res.status_int, http_resp)\n\t\treturn res
"""

description = """
Modify the get_api_response_ext method to introduce a Missing If Construct Plus Statements (MIFS) fault. The function should fail due to removing the check if headers is None:, causing potential TypeError exceptions when headers is None and iteriting over it.
"""

completion = client.chat.completions.create(
    model=model,
    temperature=0.15, # Lower temperature for more deterministic output
    messages=[
        {
            "role": "system",
            "content": (
                "You are given a Python code snippet. Your job is to classify whether it contains a bug. "
                "If it does, describe the bug and attempt a fix. If not, state that it is correct. "
                "Return your response as a structured JSON matching the provided schema. "
                "Do not explain or justify in natural language outside the JSON. "
                "Do not assume the answer is known. Use your reasoning to classify the code."
            )
        },
        {
            "role": "user",
            "content": f"The code snippet: {code_snippet}\nShort description: {description}"
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "code-diagnostic",
            "schema": json_schema
        },
    },
)

print(completion.choices[0].message.content)


{"has_bug": "has_bug", "bug_type": "Missing If Construct Plus Statements (MIFS)", "fixability": "not_fixable", "tested_functions": ["get_api_response_ext"] , "rationale": "The function is missing an if statement to check if headers is None, which could lead to a TypeError when headers is None. This is a MIFS fault because the code does not include the if condition, and the fix is not possible to implement."}
